In [1]:
import pandas as pd
import unidecode
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bapti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
gmail = pd.read_csv("data/gmail_oscar.CSV")

zimbra = pd.read_csv("data/zimbra_oscar.CSV")

In [3]:
datasets = [gmail]

In [4]:
def text_cleaning(text):
    text = unidecode.unidecode(text)
    text = text.lower()
    text = re.sub(r"""[.,"'-:!;]""", '', text)
    text = re.sub('\s+', ' ', text)
    text = text.split(' ')
    return text

In [ ]:
def stop_words_french(text):
    # print("avant de retirer les stopwords : ", len(text), " mots")
    stop_words = stopwords.words('french')
    text = [word for word in text if word not in stopwords.words('french') and len(word)>1]
    # print("Après avoir retiré les stopwords : ", len(text), " mots")
    return text

def stop_words_english(text):
    # print("avant de retirer les stopwords : ", len(text), " mots")
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stopwords.words('english') and len(word)>1]
    # print("Après avoir retiré les stopwords : ", len(text), " mots")
    return text

In [5]:
def lemmatization(nlp, texte):
    i = 0
    # On regarde chaque mot dans le texte
    # Chaque mot a le numéro i
    for mot in texte:
        # on va lemmatizer
        doc = nlp(mot)
        for token in doc:
            texte[i] = token.lemma_
        i += 1
            
    return texte

In [6]:
def remove_https(corps):
    
    text_file = open(r'corps.txt', 'w',  encoding="utf-8")
    text_file.write(corps)
    text_file.close()

        
    final_text_file = open(r'final_corps.txt', 'w',  encoding="utf-8")
    reading_text_file = open(r'corps.txt', 'r',  encoding="utf-8")
    for line in reading_text_file:
        if "https" not in line:
            final_text_file.write(line)
            
    final_text_file.close()
    reading_text_file.close()

    with open('final_corps.txt', 'r', encoding="utf-8") as file:
        return file.read()

In [7]:
def data_cleaning(df):
    
    is_english = False
    for i in df.index:
        
        corps = remove_https(df['Corps'][i])

        corps = text_cleaning(corps)
        
        objet = text_cleaning(df['Objet'][i])

        #stop words cleaning for object
        objet = stop_words_english(objet)
        objet = stop_words_french(objet)
        
        # Lemmatization
        nlp_fr = spacy.load('fr_core_news_md')
        nlp_en = spacy.load('en_core_web_md')
        objet_fr = lemmatization(nlp_fr, objet)
        objet_en = lemmatization(nlp_en, objet)
        
        # on vérifie si l'objet a été lemmatizé en anglais ou non
        if objet_en != objet:
                is_english = True
        if is_english:
            objet = objet_en
            corps = stop_words_english(corps)
            corps = lemmatization(nlp_en, corps)
        else:
            objet = objet_fr
            corps = stop_words_french(corps)
            corps = lemmatization(nlp_fr, corps)

        df['Objet'][i] = objet
        df['Corps'][i] = corps

    return df

In [8]:
print(len(gmail),len(zimbra))

10 120


In [9]:
for df in datasets:
    df = data_cleaning(df)

C:\Users\bapti\AppData\Local\Temp\ipykernel_25200\3117150056.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Objet'][i] = objet
C:\Users\bapti\AppData\Local\Temp\ipykernel_25200\3117150056.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Corps'][i] = corps


In [10]:
gmail.head()

,Objet,Corps,De: (nom),De: (adresse),De: (type),A: (nom),A: (adresse),A: (type),Cc: (nom),Cc: (adresse),Cc: (Type),Cci: (nom),Cci: (adresse),Cci: (type),Catégories,Critère de diffusion,Importance,Informations facturation,Kilométrage
0,"[course, lundi, matin, uber]","[octobre, bientot, oscar, voici, facture, cour...",Reçu Uber,noreply@uber.com,SMTP,oscarbenoit1@gmail.com,oscarbenoit1@gmail.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
1,"[verifier, frais, dannulation]","[vouloir, repondre, audessus, ce, ligne, repon...",Uber Support,contact_b92ffc2b-70bb-4116-ab15-6363af4dc0ce@e...,SMTP,oscarbenoit1@gmail.com,oscarbenoit1@gmail.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
2,"[course, lundi, matin, uber]","[octobre, oscar, merci, davoir, utilise, uber,...",Reçu Uber,noreply@uber.com,SMTP,oscarbenoit1@gmail.com,oscarbenoit1@gmail.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
3,"[vente, flash, biere, blonde]","[decouvrer, vite, selection, jusqua, demain, s...",Saveur Bière By PerfectDraft,saveur-biere@m.perfectdraft.com,SMTP,oscarbenoit1@gmail.com,oscarbenoit1@gmail.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
4,"[alerte, mail, appartement, piece, pari]","[annoncer, correspondre, recherche, location, ...",PAP.fr,users-alertes@pap.fr,SMTP,oscarbenoit1@gmail.com,oscarbenoit1@gmail.com,SMTP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal(e),NaN,NaN
